In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb 
import requests 
import gzip 
import cv2 
import zlib 
import idx2numpy
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

In [3]:
! pip install xgboost

In [4]:
! pip install opencv-contrib-python --upgrade 

In [5]:
! pip install idx2numpy

In [6]:
def load_ubyte(url, target_file):
    r = requests.get(url)
    data = zlib.decompress(r.content, zlib.MAX_WBITS|32)
    with open(target_file, 'wb') as f:
        f. write(data)
        
# Загрузим MNIST (рукописные изображения цифр)
# MNIST очень популярен для проверки качества в задачах распознания объектов

load_ubyte("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", 
           "train_data.ubyte")

load_ubyte('http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
           'train_labels.ubyte')
load_ubyte('http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
           'test_data.ubyte')
load_ubyte('http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz',
           'test_labels.ubyte')

In [7]:
# Распакуем данные и считаем картинки в массив
train_images_file = 'train_data.ubyte'
train_labels_file = 'train_labels.ubyte'
test_images_file = 'test_data.ubyte'
test_labels_file = 'test_labels.ubyte'

train_images = idx2numpy.convert_from_file(train_images_file)
train_labels = idx2numpy.convert_from_file(train_labels_file)
test_images = idx2numpy.convert_from_file (test_images_file)
test_labels = idx2numpy.convert_from_file (test_labels_file)

np.set_printoptions(linewidth=300)
print(f'Одна картинка - это набор пикселей:')
print(*train_images[2][5:-3, 2:-4], sep='\n')

Одна картинка - это набор пикселей:
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  67 232  39   0]
[  0   0  62  81   0   0   0   0   0   0   0   0   0   0   0   0   0   0 120 180  39   0]
[  0   0 126 163   0   0   0   0   0   0   0   0   0   0   0   0   0   2 153 210  40   0]
[  0   0 220 163   0   0   0   0   0   0   0   0   0   0   0   0   0  27 254 162   0   0]
[  0   0 222 163   0   0   0   0   0   0   0   0   0   0   0   0   0 183 254 125   0   0]
[  0  46 245 163   0   0   0   0   0   0   0   0   0   0   0   0   0 198 254  56   0   0]
[  0 120 254 163   0   0   0   0   0   0   0   0   0   0   0   0  23 231 254  29   0   0]
[  0 159 254 120   0   0   0   0   0   0   0   0   0   0   0   0 163 254 216  16   0   0]
[  0 159 254  67   0   0   0   0   0   0   0   0   0  14  86 178 248 254  91   0   0   0]
[  0 159 254  85   0   0   0  47  49 116 144 150 241 243 234 179 241 252  40   0   0   0]
[  0 150 253 237 207 207 207 253 254 250 240 198 143  91  28   5

In [8]:
# Моделям удобнее работать с векторами - распакуем массив пикселей в вектор
width, height = 28, 28
pixels = width * height
train_data = pd.DataFrame(
    train_images.reshape(-1, pixels),
    columns=[f'pixel_{i}' for i in range(pixels)]
)
test_data = pd.DataFrame(
    test_images.reshape (-1, pixels),
    columns=[f'pixel_{i}' for i in range(pixels)]
)
# Модель будет учиться по меткам, которые ей отдадут
train_data['label'] = train_labels

In [11]:
# Обучаем модель
# Сначала приводим значения пикселей к схожему масштабу
scaler = StandardScaler()
X_std = scaler.fit_transform(train_data.values[:, 1:])
test_std = scaler.fit_transform(test_data.values)

y_train = train_data.label
# X_valid будет использован, чтобы досрочно завершить обучение модели, 
# если ее качество перестанет улучшаться
X_train, X_valid, y_train, y_valid = train_test_split(
    X_std,
    y_train, 
    test_size=0.1, 
    random_state=42)

param_list = [
    ("eta", 0.08),
    ("max_depth", 6),
    ("subsample", 0.8),
    ("colsample_bytree", 0.8),
    ("objective", "multi:softmax"),
    ("custom_metric", "merror"), 
    ("alpha", 8), 
    ("lambda", 2),
    ("num_class", 10)]

n_rounds = 600
# Если за последние 10 раундов модель не улучшается на валидации, прекратить обучение
early_stopping = 10

d_train = xgb.DMatrix(X_train, label=y_train)
d_val = xgb.DMatrix(X_valid, label=y_valid)
eval_list = [(d_train, "train"), (d_val, "validation")]
bst = xgb.train(param_list, d_train, n_rounds, evals=eval_list, early_stopping_rounds=early_stopping, verbose_eval=True)

/Users/jmyh/anaconda3/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [14:44:13] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "custom_metric" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-mlogloss:1.95302	validation-mlogloss:1.95458
[1]	train-mlogloss:1.68514	validation-mlogloss:1.68652
[2]	train-mlogloss:1.48791	validation-mlogloss:1.48967
[3]	train-mlogloss:1.33346	validation-mlogloss:1.33582
[4]	train-mlogloss:1.20404	validation-mlogloss:1.20621
[5]	train-mlogloss:1.09731	validation-mlogloss:1.10023
[6]	train-mlogloss:0.99835	validation-mlogloss:1.00144
[7]	train-mlogloss:0.90587	validation-mlogloss:0.90868
[8]	train-mlogloss:0.82745	validation-mlogloss:0.83029
[9]	train-mlogloss:0.76061	validation-mlogloss:0.76385
[10]	train-mlogloss:0.69857	validation-mlogloss:0.70204
[11]	train-mlogloss:0.64172	validation-mlogloss:0.64533
[12]	train-mlogloss:0.59109	validation-mlogloss:0.59476
[13]	train-mlogloss:0.54306	validation-mlogloss:0.54660
[14]	train-mlogloss:0.49846	validation-mlogloss:0.50179
[15]	train-mlogloss:0.45924	validation-mlogloss:0.46239
[16]	train-mlogloss:0.42418	validation-mlogloss:0.42730
[17]	train-mlogloss:0.39505	validation-mlogloss:0.39834
[1

In [14]:
def imshow(img):
    import cv2
    import IPython
    from ipywidgets import widgets, HBox
    _, ret = cv2.imencode('.jpg', img)
    i = IPython.display.Image(data=ret, width=200, height=200)
    IPython.display.display(i)

In [16]:
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.9/214.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: comm
    Found existing installation: comm 0.1.2
    Uninstalling comm-0.1.2:
      Successfully uninstalled comm-0.1.2


In [19]:
offset, n_show = 5, 10
for image, label in zip(
    test_data.values.reshape(-1, width, height)[offset:offset+n_show], 
    test_labels[offset:offset+n_show]):
    imshow(image)
    print(f'Опознано число: {label}')

Опознано число: 1


Опознано число: 4


Опознано число: 9


Опознано число: 5


Опознано число: 9


Опознано число: 0


Опознано число: 6


Опознано число: 9


Опознано число: 0


Опознано число: 1
